### Data preparations

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

In [2]:
!pip install neptune lightning pytorchvideo

  Obtaining dependency information for neptune from https://files.pythonhosted.org/packages/82/6c/543a4d807cc0b2e03ec50f74d2ddf150903281e3fd5e29d9ff7ac9d0e658/neptune-1.8.5-py3-none-any.whl.metadata
  Obtaining dependency information for lightning from https://files.pythonhosted.org/packages/9e/8a/9642fdbdac8de47d68464ca3be32baca3f70a432aa374705d6b91da732eb/lightning-2.1.2-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.8/61.8 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.7/132.7 kB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Obtaining dependency information for boto3>=1.28.0 from https://files.pythonhosted.org/packages/45/78/e95de33d8850429f6740763eb2d5350c02fd39e916f2ddb69fe649d25125/boto3-1.29.6-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Obtaining dependency information for av from https://files.pythonhost

In [3]:
import neptune

run = neptune.init_run(
    project="afonchikovd585/Cells-classification",
    api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiJkMmY4NmNkZC05OTJlLTQwZGQtOTAwZC1kYzU1MTUwMzRjMzYifQ==",
)  # your credentials

/tmp/ipykernel_47/332987217.py:3: NeptuneWarning: The following monitoring options are disabled by default in interactive sessions: 'capture_stdout', 'capture_stderr', 'capture_traceback', and 'capture_hardware_metrics'. To enable them, set each parameter to 'True' when initializing the run. The monitoring will continue until you call run.stop() or the kernel stops. Also note: Your source files can only be tracked if you pass the path(s) to the 'source_code' argument. For help, see the Neptune docs: https://docs.neptune.ai/logging/source_code/
  run = neptune.init_run(


https://app.neptune.ai/afonchikovd585/Cells-classification/e/CEL-5


In [4]:
from pytorchvideo.data import LabeledVideoDataset, make_clip_sampler, labeled_video_dataset
from pytorchvideo.transforms import (
    ApplyTransformToKey,
    Normalize,
    RandomShortSideScale,
    UniformTemporalSubsample,
    Permute
)
from torchvision.transforms import (
    Compose,
    Lambda,
    RandomCrop,
    RandomHorizontalFlip,
    Resize
)
from torchvision.transforms._transforms_video import (
    CenterCropVideo,
    NormalizeVideo
)

/opt/conda/lib/python3.10/site-packages/torchvision/transforms/functional_tensor.py:5: UserWarning: The torchvision.transforms.functional_tensor module is deprecated in 0.15 and will be **removed in 0.17**. Please don't rely on it. You probably just need to use APIs in torchvision.transforms.functional or in torchvision.transforms.v2.functional.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/transforms/_functional_video.py:6: UserWarning: The 'torchvision.transforms._functional_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms.functional' module instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/transforms/_transforms_video.py:22: UserWarning: The 'torchvision.transforms._transforms_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms' module instead.
  warnings.warn(


In [5]:
video_transforms = Compose([
    ApplyTransformToKey(key='video',
    transform=Compose([
        UniformTemporalSubsample(20),
        Normalize((0.45, 0.45, 0.45),(0.225, 0.225, 0.225)),
        # RandomShortSideScale(min_size=248, max_size=256),
        # CenterCropVideo(224),
        RandomHorizontalFlip(p=0.5),
    ]),
    ),
])

In [6]:
import pandas as pd
import numpy as np
import os
import shutil

In [7]:
from torch.utils.data import DataLoader
from pytorch_lightning.callbacks import ModelCheckpoint, LearningRateMonitor

In [8]:
dataset_path = "/kaggle/input/cells-dataset/dataset"

In [9]:
os.chdir(dataset_path)
os.chdir("..")

In [10]:
dataframe = pd.read_csv("DataFrame.csv", index_col=0)
dataframe = dataframe.sample(frac=1)
ratio = 0.8
train_size = int(dataframe.shape[0] * ratio)
train_data = dataframe[0:train_size]
test_data = dataframe[train_size:]
train_data, test_data

(            files  erythrocytes  lymphocytes  high_erythrocytes  \
 1120  traffic1121          4989           83                  0   
 711    traffic712          5110          225                  1   
 853    traffic854          4949          266                  0   
 264    traffic265          5107          235                  1   
 552    traffic553          4919          155                  0   
 ...           ...           ...          ...                ...   
 156    traffic157          5021          241                  1   
 1017  traffic1018          4902          213                  0   
 963    traffic964          4920          178                  0   
 369    traffic370          5019          294                  1   
 611    traffic612          4999          222                  0   
 
       high_lymphocytes  blur  noise  
 1120                 0  10.0      1  
 711                  1   0.0      1  
 853                  1   7.5      1  
 264                  1   

In [11]:
class CustomDataset(LabeledVideoDataset):
    def __init__(self, dataset_path, dataframe, target_name, transforms, clip_sampler_type='random', clip_duration=1):
      df = dataframe.reset_index()
      paths = []
      for i, file_name in enumerate(df['files']):
          temp_dict = df.iloc[i].to_dict()
          temp_dict['label'] = df[target_name][i]
          temp_dict.pop('files')
          temp_dict.pop('index')
          paths.append((f"{dataset_path}/{file_name}", temp_dict))
      super().__init__(labeled_video_paths=paths,
                       clip_sampler=make_clip_sampler(clip_sampler_type, clip_duration),
                       transform=transforms, decode_audio=False)

In [ ]:
train_dataset = CustomDataset(dataset_path=dataset_path, dataframe=train_data,
                              target_name='high_erythrocytes',
                              transforms=video_transforms)

In [ ]:
train_dataset.num_videos

In [12]:
checkpoint_callback = ModelCheckpoint(monitor="val/loss", dirpath="checkpoints", filename="file", save_last=True)
lr_monitor = LearningRateMonitor(logging_interval="epoch")

In [13]:
class CurriculumTrainer():
    def set_difficulty(self, dataframe, target_cells, target_name, alpha, beta, gamma):
        # Calculating distances between mean and target value
        df = dataframe.copy(deep=True)
        df['distances'] = df[target_cells].mean() - df[target_cells]

        # Normalizing blur and distances
        #df['blur'] = (df['blur'] - df['blur'].mean()) / df['blur'].std()
        df['distances'] = (df['distances'] - df['distances'].mean()) / df['distances'].std()

        # Calculating and normalizing difficulty
        df['difficulty'] = df['blur'] * alpha + df['noise'] * beta + df['distances'] * gamma
        df['difficulty'] = (df['difficulty'] - df['difficulty'].min()) / (df['difficulty'].max() - df['difficulty'].min())
        return df

    def evaluate_competence(self, max_epochs, current_epoch, c0, p):
        return min(1, (current_epoch*((1-c0**p)/max_epochs)+c0**p)**(1/p))

    def fit(self, model, dataframe, target_cells, target_name, max_epochs, c0, p, alpha, beta, gamma):
        dataframe = self.set_difficulty(dataframe, target_cells, target_name, alpha, beta, gamma)
        self.competence = c0
        seed_everything(0)
        for epoch in range(1, max_epochs+1):
            selected_data = dataframe[dataframe.difficulty < self.competence]
            dataset = CustomDataset(dataset_path=dataset_path, dataframe=selected_data,
                                    target_name=target_name,
                                    transforms=video_transforms)
            loader = DataLoader(dataset, batch_size=model.batch_size, num_workers=model.numworkers, pin_memory=True)
            print(f"-----------------------\nEpoch {epoch}, competence = {self.competence}, dataset size = {dataset.num_videos}")
            run["model/competence"].append(self.competence)
            run["train/dataset_size"].append(dataset.num_videos)
            self.trainer = Trainer(max_epochs=1,
                                   precision='16-mixed',
                                   accumulate_grad_batches=2,
                                   enable_progress_bar=False,
                                   enable_model_summary=False,
                                   num_sanity_val_steps=0,
                                   callbacks=[lr_monitor, checkpoint_callback])
            self.trainer.fit(model, loader)
            self.competence = self.evaluate_competence(max_epochs, epoch, c0, p)

    def validate(self, model):
#         dataset = CustomDataset(dataset_path=dataset_path, dataframe=test_data,
#                                 target_name=model.target_label,
#                                 transforms=video_transforms)
#         loader = DataLoader(dataset, batch_size=model.batch_size, num_workers=model.numworkers, pin_memory=True)
#         self.trainer.validate(model, loader)
        self.trainer.validate(model)

    def test(self, model):
#         dataset = CustomDataset(dataset_path=dataset_path, dataframe=test_data,
#                                 target_name=model.target_label,
#                                 transforms=video_transforms)
#         loader = DataLoader(dataset, batch_size=model.batch_size, num_workers=model.numworkers, pin_memory=True)
#         self.trainer.test(model, loader)
        self.trainer.test(model)

### Model

In [14]:
import torch
import torch.nn as nn
from pytorch_lightning import LightningModule, seed_everything, Trainer
from torch.optim.lr_scheduler import CosineAnnealingLR
from sklearn.metrics import classification_report
import torchmetrics

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [15]:
class TestModel(LightningModule):
    def __init__(self, target_label, num_classes=2):
        super(TestModel, self).__init__()
        # model architecture
        self.video_model = torch.hub.load("facebookresearch/pytorchvideo", "efficient_x3d_xs", pretrained=True)
        self.relu = nn.ReLU()
        self.linear = nn.Linear(400, num_classes)

        self.lr = 1e-3
        self.batch_size = 8
        self.numworkers = 0
        # evaluation metric
        self.metric = torchmetrics.Accuracy(task='multiclass', num_classes=num_classes)
        # loss function
        self.criterion = nn.CrossEntropyLoss()
        # helpers
        self.target_label = target_label
        self.training_step_outputs = []
        self.validation_step_outputs = []
        self.testing_step_outputs = []

    def forward(self, x):
        x = self.video_model(x)
        x = self.relu(x)
        x = self.linear(x)
        return x

    def configure_optimizers(self):
        opt = torch.optim.AdamW(params=self.parameters(), lr=self.lr)
        scheduler = CosineAnnealingLR(opt, T_max=10, eta_min=1e-6, last_epoch=-1)
        return {'optimizer': opt, 'lr_scheduler': scheduler}

    # def train_dataloader(self):
    #     dataset = CustomDataset(dataset_path=dataset_path, dataframe=train_data,
    #                           target_name=self.target_label,
    #                           transforms=video_transforms)
    #     loader = DataLoader(dataset, batch_size=self.batch_size, num_workers=self.numworkers, pin_memory=True)
    #     return loader

    def training_step(self, batch, batch_idx):
        video, label = batch['video'], batch['label']
        out = self.forward(video)
        loss = self.criterion(out, label)
        metric = self.metric(out, label.to(torch.int64))
        self.training_step_outputs.append({'loss': loss, 'metric': metric})
        return {'loss': loss, 'metric': metric}

    def on_train_epoch_end(self):
        outputs = self.training_step_outputs
        loss = torch.stack([x['loss'] for x in outputs]).mean().cpu().detach().numpy().round(2)
        metric = torch.stack([x['metric'] for x in outputs]).mean().cpu().detach().numpy().round(2)
        print({'loss': loss, 'metric': metric})
        self.training_step_outputs = []
        self.log('train/loss', loss)
        self.log('train/metric', metric)
        run["train/loss"].append(loss)
        run["train/metric"].append(metric)

    def val_dataloader(self):
        dataset = CustomDataset(dataset_path=dataset_path, dataframe=test_data,
                              target_name=self.target_label,
                              transforms=video_transforms)
        loader = DataLoader(dataset, batch_size=self.batch_size, num_workers=self.numworkers, pin_memory=True)
        return loader

    def validation_step(self, batch, batch_idx):
        video, label = batch['video'], batch['label']
        out = self.forward(video)
        loss = self.criterion(out, label)
        metric = self.metric(out, label.to(torch.int64))
        self.validation_step_outputs.append({'loss': loss, 'metric': metric})
        return {'loss': loss, 'metric': metric}

    def on_validation_epoch_end(self):
        outputs = self.validation_step_outputs
        loss = torch.stack([x['loss'] for x in outputs]).mean().cpu().detach().numpy().round(2)
        metric = torch.stack([x['metric'] for x in outputs]).mean().cpu().detach().numpy().round(2)
        self.validation_step_outputs = []
        self.log('val/loss', loss)
        self.log('val/metric', metric)
        run["val/loss"].append(loss)
        run["val/metric"].append(metric)

    def test_dataloader(self):
        dataset = CustomDataset(dataset_path=dataset_path, dataframe=test_data,
                              target_name=self.target_label,
                              transforms=video_transforms)
        loader = DataLoader(dataset, batch_size=self.batch_size, num_workers=self.numworkers, pin_memory=True)
        return loader

    def test_step(self, batch, batch_idx):
        video, label = batch['video'], batch['label']
        out = self.forward(video)
        self.testing_step_outputs.append({'label': label, 'pred': out})
        return {'label': label, 'pred': out}

    def on_test_epoch_end(self):
        outputs = self.testing_step_outputs
        label = torch.cat([x['label'] for x in outputs]).cpu().detach().numpy()
        pred = torch.cat([x['pred'].argmax(dim=1) for x in outputs]).cpu().detach().numpy()
        self.testing_step_outputs = []
        print(classification_report(label, pred))

In [16]:
model = TestModel(num_classes=2, target_label='high_erythrocytes')

Downloading: "https://github.com/facebookresearch/pytorchvideo/zipball/main" to /root/.cache/torch/hub/main.zip
Downloading: "https://dl.fbaipublicfiles.com/pytorchvideo/model_zoo/kinetics/efficient_x3d_xs_original_form.pyth" to /root/.cache/torch/hub/checkpoints/efficient_x3d_xs_original_form.pyth
100%|██████████| 14.8M/14.8M [00:00<00:00, 43.0MB/s]


In [17]:
params = {
    "n_epochs": 100,
    "c0": 0.01,
    "p": 2,
    "alpha": 0.34,
    "beta": 0.33,
    "gamma": 0.33
}
run["model/parameters"] = params
run["model/architecture"] = "efficient_x3d_xs"

In [18]:
os.chdir("/kaggle/working/")

In [19]:
# Run this for trainer without curriculum
seed_everything(0)
trainer = Trainer(max_epochs=params["n_epochs"],
                  precision='16-mixed',
                  accumulate_grad_batches=2,
                  enable_progress_bar=True,
                  num_sanity_val_steps=0,
                  callbacks=[lr_monitor, checkpoint_callback])

dataset = CustomDataset(dataset_path=dataset_path, dataframe=train_data,
                      target_name=model.target_label,
                      transforms=video_transforms)
loader = DataLoader(dataset, batch_size=model.batch_size, num_workers=model.numworkers, pin_memory=True)
trainer.fit(model, loader)

/opt/conda/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=3` in the `DataLoader` to improve performance.
/opt/conda/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=3` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

{'loss': 0.8, 'metric': 0.5}


Validation: |          | 0/? [00:00<?, ?it/s]

...

{'loss': 0.21, 'metric': 0.91}


Validation: |          | 0/? [00:00<?, ?it/s]

{'loss': 0.22, 'metric': 0.91}


Validation: |          | 0/? [00:00<?, ?it/s]

{'loss': 0.26, 'metric': 0.88}


In [37]:
# Run this for curriculum learning
trainer = CurriculumTrainer()
trainer.fit(model, train_data, 'erythrocytes', 'high_erythrocytes',
            params["n_epochs"], params["c0"], params["p"],
            params["alpha"], params["beta"], params["gamma"])

In [20]:
trainer.validate(model)

/opt/conda/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=3` in the `DataLoader` to improve performance.


Validation: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Validate metric      ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         val/loss          │    0.5199999809265137     │
│        val/metric         │    0.7599999904632568     │
└───────────────────────────┴───────────────────────────┘

[{'val/loss': 0.5199999809265137, 'val/metric': 0.7599999904632568}]

In [21]:
trainer.test(model)

/opt/conda/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=3` in the `DataLoader` to improve performance.


Testing: |          | 0/? [00:00<?, ?it/s]

              precision    recall  f1-score   support

           0       0.80      0.72      0.76       123
           1       0.71      0.79      0.75       107

    accuracy                           0.75       230
   macro avg       0.75      0.75      0.75       230
weighted avg       0.76      0.75      0.75       230



[{}]

In [22]:
run.stop()

Shutting down background jobs, please wait a moment...
Done!
All 0 operations synced, thanks for waiting!
Explore the metadata in the Neptune app:
https://app.neptune.ai/afonchikovd585/Cells-classification/e/CEL-5/metadata
